Convert and store data in a js file which can be read and used as variables by javascript programs.

In [33]:
import os
import numpy as np
import pandas as pd
import json

In [34]:
labels = ['Gaze','Scan','GazeInPlace','ScanInPlace']
N_ROWS = 526
N_COLS = 658

In [35]:
def coord_transform(point):
#     h=658
#     w=526
#     h1=100
#     w1=80
#     reflection = np.array([[1,0,0],
#                            [0,-1,0],
#                            [0,0,1]])
#     translate = np.array([[1,0,w1],
#                           [0,1,0],
#                           [0,0,1]])
#     scale = np.array([[w/w1,0,0],
#                       [0,h/h1,0],
#                       [0,0,1]])
    import math
    mat = np.array([[6.575,0,526],
                    [0,-6.58,0],
                    [0,0,1]])
    vector=(*point,1)
    result = mat.dot(vector)
    return np.floor(result[0:2]).astype(int)

coord_transform([-40,-50])
coord_transform([-39.581,-68.3935])

array([265, 450])

In [44]:
processed = './processed/'
js = './js/'
all_data = {}
for filename in os.listdir(processed):
    fullpath = processed+filename
    print(fullpath)
    dataFrame = pd.read_csv(fullpath,#index_col=13,
                    dtype={'no':np.int32, 'posx':np.float64,'posy':np.float64,'posz':np.float64,
                           'movedirx':np.float64,'movediry':np.float64,'movedirz':np.float64,
                           'viewdirx':np.float64,'viewdiry':np.float64,'viewdirz':np.float64,
                           'deltapos':np.float64,'deltamovedir':np.float64,'deltaviewdir':np.float64,'label':np.int32})
#     print(dataFrame.head())
    
    # 四类行为
    playerData = {}
    for i in range(1,5):
        bins = np.zeros((N_ROWS,N_COLS))
        subDf = dataFrame[dataFrame['label']==i]
#         print(subDf.shape)
        for idx,row in subDf.iterrows():
            pixel = coord_transform([row['posx'],row['posy']])
            bins[pixel[0],pixel[1]] += 1
            
        # convert data to js object
        dataArray = []
        max = 0
        for (row,col),value in np.ndenumerate(bins):
            if value == 0: continue
            max = value if value > max else max
            dataArray.append({'x':row,'y':col,'value':value})
        dataobj = {'max':max,'data':dataArray}
        
        # append to parent object
        playerData[labels[i-1]] = dataobj
        # update value
        if labels[i-1] in all_data:
            maxvalue = all_data[labels[i-1]]['max']
            if dataobj['max'] > maxvalue:
                all_data[labels[i-1]]['max'] = dataobj['max']
            all_data[labels[i-1]]['data'].extend(dataobj['data'])
        else:
            all_data[labels[i-1]] = dataobj
        
    #write to a js file
#     print(playerData)
    jsonobj = json.dumps(playerData)
    filename = filename[:-4]
    content = 'var player{} = {};'.format(filename, jsonobj)
    target_filename = js+'player'+filename+'.js'
    with open(target_filename,'w') as fp:
        fp.write(content)
    
# write all in one
# print(all_data)
jsonobj = json.dumps(all_data)
content = 'var all = {};'.format(jsonobj)
all_filename = js+'all'+'.js'
with open(all_filename,'w') as fp:
    fp.write(content)
    
print('Finished!')

./processed/10.csv
./processed/11.csv
./processed/12.csv
./processed/13.csv
./processed/14.csv
./processed/16.csv
./processed/17.csv
./processed/18.csv
./processed/19.csv
./processed/20.csv
./processed/21.csv
./processed/22.csv
./processed/23.csv
./processed/24.csv
./processed/25.csv
./processed/3.csv
./processed/4.csv
./processed/5.csv
./processed/6.csv
./processed/7.csv
./processed/8.csv
./processed/9.csv
./processed/x1.csv
./processed/x2.csv
./processed/x3.csv
./processed/x4.csv
Finished!
